In [1]:
from tensorflow.keras.applications.resnet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os


In [2]:
prototxtPath=os.path.sep.join([r'C:\Users\Ranjith ch\Mask detector\face detector','deploy.prototxt'])
weightsPath=os.path.sep.join([r'C:\Users\Ranjith ch\Mask detector\face detector','res10_300x300_ssd_iter_140000.caffemodel'])

In [3]:
weightsPath

'C:\\Users\\Ranjith ch\\Mask detector\\face detector\\res10_300x300_ssd_iter_140000.caffemodel'

In [4]:
net=cv2.dnn.readNet(prototxtPath,weightsPath)

In [5]:
model=load_model(r'C:\Users\Ranjith ch\Mask detector\ResNet15V2_model.model')

In [7]:
image.shape

(562, 1000, 3)

In [6]:
image=cv2.imread(r'C:\Users\Ranjith ch\Mask detector\example_05.jpg')

In [8]:
(h,w)=image.shape[:2]

In [9]:
blob=cv2.dnn.blobFromImage(image,1.0,(300,300),(104.0,177.0,123.0))

In [10]:
blob

array([[[[  73.,   71.,   70., ...,  147.,   97.,   74.],
         [  73.,   72.,   68., ...,  143.,   81.,   66.],
         [  73.,   72.,   69., ...,  142.,   81.,   44.],
         ...,
         [  71.,   62.,  -85., ...,  151.,  147.,  146.],
         [  75.,   21.,  -85., ...,  149.,  145.,  146.],
         [  71.,   -6.,  -90., ...,  146.,  145.,  147.]],

        [[   0.,   -2.,   -3., ...,   73.,   26.,    5.],
         [   0.,   -1.,   -5., ...,   69.,   10.,   -3.],
         [   0.,   -1.,   -4., ...,   68.,   10.,  -26.],
         ...,
         [  -4.,  -16., -167., ...,   75.,   72.,   70.],
         [   0.,  -56., -165., ...,   74.,   71.,   72.],
         [  -6.,  -82., -170., ...,   74.,   70.,   71.]],

        [[  54.,   52.,   51., ...,  123.,   74.,   55.],
         [  54.,   53.,   49., ...,  119.,   58.,   46.],
         [  54.,   53.,   50., ...,  118.,   58.,   23.],
         ...,
         [  49.,   35., -116., ...,  123.,  118.,  116.],
         [  53.,   -5., -1

In [11]:
net.setInput(blob)

In [12]:
detections=net.forward()

In [13]:
detections

array([[[[0.        , 1.        , 0.96808237, ..., 0.36397934,
          0.163156  , 0.50138706],
         [0.        , 1.        , 0.956191  , ..., 0.67439127,
          0.9611798 , 0.828423  ],
         [0.        , 1.        , 0.91764534, ..., 0.12592609,
          0.66664743, 0.24871598],
         ...,
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ]]]], dtype=float32)

In [14]:
for i in range(0,detections.shape[2]):
    confidence=detections[0,0,i,2]
    
    
    if confidence>0.5:
        #we need the X,Y coordinates
        box=detections[0,0,i,3:7]*np.array([w,h,w,h])
        (startX,startY,endX,endY)=box.astype('int')
        
        #ensure the bounding boxes fall within the dimensions of the frame
        (startX,startY)=(max(0,startX),max(0,startY))
        (endX,endY)=(min(w-1,endX), min(h-1,endY))
        
        
        #extract the face ROI, convert it from BGR to RGB channel, resize it to 224,224 and preprocess it
        face=image[startY:endY, startX:endX]
        face=cv2.cvtColor(face,cv2.COLOR_BGR2RGB)
        face=cv2.resize(face,(224,224))
        face=img_to_array(face)
        face=preprocess_input(face)
        face=np.expand_dims(face,axis=0)
        
        (mask,withoutMask)=model.predict(face)[0]
        
        #determine the class label and color we will use to draw the bounding box and text
        label='Mask' if mask>withoutMask else 'No Mask'
        color=(255,0,0) if label=='Mask' else (0,0,255)
        
        #include the probability in the label
        label="{}: {:.2f}%".format(label,max(mask,withoutMask)*100)
        
        #display the label and bounding boxes
        cv2.putText(image,label,(startX,startY-10),cv2.FONT_HERSHEY_SIMPLEX,0.45,color,2)
        cv2.rectangle(image,(startX,startY),(endX,endY),color,2)
        
        
        
cv2.imshow("OutPut",image)
cv2.waitKey(0)
cv2.destroyAllWindows()